In [1]:
import pandas as pd
import re
import datetime
import numpy as np
import time
from gpt3 import try_gpt

# 2022 Data Preprocess

In [2]:
RELEVANT_COLUMNS = ['home_team', 'away_team', 'posteam', 'posteam_type', 'defteam', 'side_of_field', 'yardline_100', 'quarter_seconds_remaining', 'half_seconds_remaining', 'game_seconds_remaining', 'game_half', 'quarter_end', 'drive', 'sp', 'qtr', 'down', 'goal_to_go', 'time', 'yrdln', 'ydstogo', 'ydsnet', 'play_type', 'yards_gained', 'shotgun', 'no_huddle', 'qb_dropback', 'qb_kneel', 'qb_spike', 'qb_scramble', 'pass_length', 'pass_location', 'air_yards', 'yards_after_catch', 'run_location', 'run_gap', 'field_goal_result', 'kick_distance', 'extra_point_result', 'two_point_conv_result', 'away_timeouts_remaining', 'td_team', 'posteam_timeouts_remaining', 'defteam_timeouts_remaining', 'total_home_score', 'total_away_score', 'posteam_score', 'defteam_score', 'score_differential', 'posteam_score_post', 'defteam_score_post', 'score_differential_post', 'punt_blocked', 'first_down_rush', 'first_down_pass', 'first_down_penalty', 'third_down_converted', 'third_down_failed', 'fourth_down_converted', 'fourth_down_failed', 'incomplete_pass', 'touchback', 'interception', 'punt_inside_twenty', 'punt_in_endzone', 'punt_out_of_bounds', 'punt_downed', 'punt_fair_catch', 'kickoff_inside_twenty', 'kickoff_in_endzone', 'kickoff_out_of_bounds', 'kickoff_downed', 'kickoff_fair_catch', 'fumble_forced', 'fumble_not_forced', 'fumble_out_of_bounds', 'solo_tackle', 'safety', 'penalty', 'tackled_for_loss', 'fumble_lost', 'own_kickoff_recovery', 'own_kickoff_recovery_td', 'qb_hit', 'rush_attempt', 'pass_attempt', 'sack', 'touchdown', 'pass_touchdown', 'rush_touchdown', 'return_touchdown', 'extra_point_attempt', 'two_point_attempt', 'field_goal_attempt', 'kickoff_attempt', 'punt_attempt', 'fumble', 'complete_pass', 'assist_tackle', 'lateral_reception', 'lateral_rush', 'lateral_return', 'lateral_recovery', 'passer_player_id', 'passer_player_name', 'receiver_player_id', 'receiver_player_name', 'rusher_player_id', 'rusher_player_name', 'lateral_receiver_player_id', 'lateral_receiver_player_name', 'lateral_rusher_player_id', 'lateral_rusher_player_name', 'lateral_sack_player_id', 'lateral_sack_player_name', 'interception_player_id', 'interception_player_name', 'lateral_interception_player_id', 'lateral_interception_player_name', 'punt_returner_player_id', 'punt_returner_player_name', 'lateral_punt_returner_player_id', 'lateral_punt_returner_player_name', 'kickoff_returner_player_name', 'kickoff_returner_player_id', 'lateral_kickoff_returner_player_id', 'lateral_kickoff_returner_player_name', 'punter_player_id', 'punter_player_name', 'kicker_player_name', 'kicker_player_id', 'own_kickoff_recovery_player_id', 'own_kickoff_recovery_player_name', 'blocked_player_id', 'blocked_player_name', 'tackle_for_loss_1_player_id', 'tackle_for_loss_1_player_name', 'tackle_for_loss_2_player_id', 'tackle_for_loss_2_player_name', 'qb_hit_1_player_id', 'qb_hit_1_player_name', 'qb_hit_2_player_id', 'qb_hit_2_player_name', 'forced_fumble_player_1_team', 'forced_fumble_player_1_player_id', 'forced_fumble_player_1_player_name', 'forced_fumble_player_2_team', 'forced_fumble_player_2_player_id', 'forced_fumble_player_2_player_name', 'solo_tackle_1_team', 'solo_tackle_2_team', 'solo_tackle_1_player_id', 'solo_tackle_2_player_id', 'solo_tackle_1_player_name', 'solo_tackle_2_player_name', 'assist_tackle_1_player_id', 'assist_tackle_1_player_name', 'assist_tackle_1_team', 'assist_tackle_2_player_id', 'assist_tackle_2_player_name', 'assist_tackle_2_team', 'assist_tackle_3_player_id', 'assist_tackle_3_player_name', 'assist_tackle_3_team', 'assist_tackle_4_player_id', 'assist_tackle_4_player_name', 'assist_tackle_4_team', 'pass_defense_1_player_id', 'pass_defense_1_player_name', 'pass_defense_2_player_id', 'pass_defense_2_player_name', 'fumbled_1_team', 'fumbled_1_player_id', 'fumbled_1_player_name', 'fumbled_2_player_id', 'fumbled_2_player_name', 'fumbled_2_team', 'fumble_recovery_1_team', 'fumble_recovery_1_yards', 'fumble_recovery_1_player_id', 'fumble_recovery_1_player_name', 'fumble_recovery_2_team', 'fumble_recovery_2_yards', 'fumble_recovery_2_player_id', 'fumble_recovery_2_player_name', 'return_team', 'return_yards', 'penalty_team', 'penalty_player_id', 'penalty_player_name', 'penalty_yards', 'replay_or_challenge', 'replay_or_challenge_result', 'penalty_type', 'defensive_two_point_attempt', 'defensive_two_point_conv', 'defensive_extra_point_attempt', 'defensive_extra_point_conv']

In [3]:
pbp_data_22 = pd.read_csv("./pbp_data/regular_season/reg_pbp_2022.csv")

/var/folders/08/6rx4zjtn10557fvbrhbl5ykw0000gn/T/ipykernel_44347/813836328.py:1: DtypeWarning: Columns (179,180,182,183,189,190,193,194,197,198,203,204,205,206,218,219,220,243,244,245,253,254,255,260,262,263,283,284,301) have mixed types. Specify dtype option on import or set low_memory=False.
  pbp_data_22 = pd.read_csv("./pbp_data/regular_season/reg_pbp_2022.csv")


In [4]:
highlights22 = pd.read_csv("./transcripts_2022.csv")
highlights22 = highlights22[~highlights22["interval"].isna()]

In [5]:
def process_inning_time(time_str):
    quarter = time_str[0]
    if quarter == "O":
        time_remaining = 0
    else:
        time_remaining = (4 - int(quarter)) * 900
    times = re.findall("\d?\d:\d\d",time_str)
    timedeltas = pd.to_timedelta(pd.to_datetime(times,format='%M:%S').strftime('%H:%M:%S').values)
    timelists = (timedeltas + datetime.timedelta(seconds=time_remaining)).seconds
    return pd.Interval(timelists[1],timelists[0],closed="both")

In [6]:
highlights22["intervals"] = highlights22["interval"].apply(process_inning_time)
highlights22["date"] = pd.to_datetime(highlights22["date"])
pbp_data_22["game_date"] = pd.to_datetime(pbp_data_22["game_date"])

In [7]:
merged22 = pbp_data_22.merge(highlights22,left_on=["home_team","away_team","game_date"], right_on=["home","away","date"])

In [8]:
plays22 = merged22[merged22.apply(lambda x: x["game_seconds_remaining"] in x["intervals"],axis=1)]

In [9]:
touches = plays22[plays22["touchdown"] == 1].groupby("intervals")['touchdown'].sum()

In [10]:
touchdown_index = touches[touches == 1].index

In [11]:
plays22 = plays22[(plays22["touchdown"] == 1) & (plays22["intervals"].isin(touchdown_index))]

In [12]:
X = plays22[RELEVANT_COLUMNS+["transcript"]]

In [13]:
ret_list = []
for i in range(X.shape[0]):
    ret_list += [{"json":X[RELEVANT_COLUMNS].iloc[i].replace(0,np.nan).dropna().to_json(),
                  "transcript":X["transcript"].iloc[i]}]

In [14]:
gpt22 = pd.DataFrame(ret_list)

In [15]:
group1 = gpt22[:10]
group2 = gpt22[10:20]
group3 = gpt22[20:]
group1.loc["commentary"] = group1["json"].apply(try_gpt)
time.sleep(5)
group2.loc["commentary"] = group2["json"].apply(try_gpt)
time.sleep(5)
group3.loc["commentary"] = group3["json"].apply(try_gpt)

/var/folders/08/6rx4zjtn10557fvbrhbl5ykw0000gn/T/ipykernel_44347/3730512515.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group1.loc["commentary"] = group1["json"].apply(try_gpt)
/var/folders/08/6rx4zjtn10557fvbrhbl5ykw0000gn/T/ipykernel_44347/3730512515.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group2.loc["commentary"] = group2["json"].apply(try_gpt)
/var/folders/08/6rx4zjtn10557fvbrhbl5ykw0000gn/T/ipykernel_44347/3730512515.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

In [16]:
final_sheet = pd.concat([group1,group2,group3])
final_sheet.head()

,json,transcript
0,"{""home_team"":""HOU"",""away_team"":""CLE"",""posteam""...",What a punt here by Johnson. Peoples-jones. Go...
1,"{""home_team"":""HOU"",""away_team"":""CLE"",""posteam""...",On 2nd and 10. Allen trying to catch some brea...
2,"{""home_team"":""HOU"",""away_team"":""CLE"",""posteam""...",Browns fans hoping to get a little help. That ...
3,"{""home_team"":""CHI"",""away_team"":""GB"",""posteam"":...","Fields, oh what a move to move past Nixon. And..."
4,"{""home_team"":""CHI"",""away_team"":""GB"",""posteam"":...",Hit to Montgomery. With a nice cut. And he pow...


In [17]:
final_sheet["gpt"] = final_sheet["commentary"].str.replace("\n+"," ").str.strip()
final_sheet["real_transcript"] = final_sheet["transcript"].str.replace("\n+"," ").str.strip()

KeyError: 'commentary'

In [ ]:
# final_sheet[["real_transcript","gpt"]].to_csv("blind_test_solutions.csv",index=False)

In [ ]:
shuffled = final_sheet[["real_transcript","gpt"]].apply(lambda x : x.sample(frac=1).reset_index(drop=True),axis=1)

In [ ]:
# shuffled.to_csv("blind_test.csv",index=False)